Nota generada a partir de la [liga1](https://www.dropbox.com/s/jfrxanjls8kndjp/Diferenciacion_e_Integracion.pdf?dl=0).

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -p 8786:8786 -p 8787:8787 -d palmoreck/jupyterlab_numerical:1.1.0
```

Detener el contenedor de docker:

```
docker stop jupyterlab_local
```


In [15]:
%pip install -q --user line_profiler

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install -q --user memory_profiler

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install -q --user psutil

Note: you may need to restart the kernel to use updated packages.


In [18]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

En esta nota revisamos algunas herramientas de Python para perfilamiento de código: uso de cpu y memoria como:

* Medición de tiempos con: 

    * [time](https://docs.python.org/3/library/time.html#time.time) de Python.
    * [/usr/bin/time](https://en.wikipedia.org/wiki/Time_(Unix)) de `Unix`
    * [%timeit](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) de comandos de magic.

* Perfilamiento de CPU con: `line_profiler` y `CProfile` y de memoria con: `memory_profiler`.

Utilizamos los métodos revisados en la nota [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb).

In [1]:
import math
import numpy as np

from scipy.integrate import quad
import matplotlib.pyplot as plt

### Regla compuesta del rectángulo

**Ejemplo de implantación de regla compuesta de rectángulo: usando math**

Utilizar la regla compuesta del rectángulo para aproximar la integral $\int_0^1e^{-x^2}dx$.

In [2]:
f=lambda x: math.exp(-x**2) #using math library

In [3]:
def Rcf_1(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_1 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=[mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res 

In [4]:
aprox_1=Rcf_1(f,0,1,1)
aprox_1

0.7788007830714049

In [5]:
aprox_2=Rcf_1(f,0,1,2)
aprox_2

0.7545979437721995

In [6]:
aprox_3=Rcf_1(f,0,1,10**3)
aprox_3

0.746824163469049

Y se puede evaluar el error de aproximación con el error relativo:

In [7]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [8]:
obj, err = quad(f, 0, 1)
(err_relativo(aprox_1,obj), err_relativo(aprox_2,obj), err_relativo(aprox_3,obj))

(0.04281684114646715, 0.010409158754012628, 4.1049318789768585e-08)

In [9]:
aprox_4=Rcf_1(f,0,1,10**5)
aprox_4

0.7468241328154887

In [10]:
err_relativo(aprox_4,obj)

4.099426997862257e-12

Al menos para este ejemplo con $10^5$ nodos parece ser numéricamente estable... 

## Uso de line_profiler en python: %lprun

In [11]:
def Rcf_2(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_2 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=(mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n))
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

In [12]:
def Rcf_3(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_3 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=(mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n))
    return h_hat*sum((f(node) for node in nodes))

In [13]:
def Rcf_4(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_4 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=(mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n))
    return h_hat*sum((math.exp(-node**2) for node in nodes))

In [14]:
%load_ext line_profiler

In [15]:
%lprun -f Rcf_1 Rcf_1(f,0,1,10**5)

Timer unit: 1e-06 s

Total time: 0.287286 s
File: <ipython-input-3-8b2da2b2416f>
Function: Rcf_1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_1(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): lef

In [16]:
%lprun -f Rcf_2 Rcf_2(f,0,1,10**5)

Timer unit: 1e-06 s

Total time: 0.254116 s
File: <ipython-input-11-08e2d3f927ed>
Function: Rcf_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_2(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): le

In [17]:
%lprun -f Rcf_3 Rcf_3(f,0,1,10**5)

Timer unit: 1e-06 s

Total time: 0.183197 s
File: <ipython-input-12-e6fccc57a114>
Function: Rcf_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_3(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): le

In [18]:
%lprun -f Rcf_4 Rcf_4(0,1,10**5)

Timer unit: 1e-06 s

Total time: 0.166477 s
File: <ipython-input-13-f3d4bfb56109>
Function: Rcf_4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf_4(a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): left

In [19]:
%%file Rcf_1_b.py

import math
@profile
def Rcf_1(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_1 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=[mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res 
if __name__=="__main__":
        f=lambda x: math.exp(-x**2)
        print(Rcf_1(f,0,1,10**5))

Writing Rcf_1_b.py


**Ejemplo line_profiler desde la línea de comandos:**

In [20]:
%%bash
/home/miuser/.local/bin/kernprof -l -v Rcf_1_b.py

0.7468241328154887
Wrote profile results to Rcf_1_b.py.lprof
Timer unit: 1e-06 s

Total time: 0.230992 s
File: Rcf_1_b.py
Function: Rcf_1 at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           @profile
     4                                           def Rcf_1(f,a,b,n):
     5                                               """
     6                                               Compute numerical approximation using rectangle or mid-point method in 
     7                                               an interval.
     8                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     9                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    10                                               Args:
    11                                                   f (lambda expression): l

## Uso de %timeit

In [21]:
%timeit?

Docstring:
Time execution of a Python statement or expression

Usage, in line mode:
  %timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] statement
or in cell mode:
  %%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] setup_code
  code
  code...

Time execution of a Python statement or expression using the timeit
module.  This function can be used both as a line and cell magic:

- In line mode you can time a single-line statement (though multiple
  ones can be chained with using semicolons).

- In cell mode, the statement in the first line is used as setup code
  (executed but not timed) and the body of the cell is timed.  The cell
  body has access to any variables created in the setup code.

Options:
-n<N>: execute the given statement <N> times in a loop. If <N> is not
provided, <N> is determined so as to get sufficient accuracy.

-r<R>: number of repeats <R>, each consisting of <N> loops, and take the
best result.
Default: 7

-t: use time.time to measure the time, which is the default on Unix.
This f

In [22]:
(err_relativo(Rcf_3(f,0,1,10**5),obj),err_relativo(Rcf_4(0,1,10**5),obj))

(4.099426997862257e-12, 4.099426997862257e-12)

In [23]:
%timeit -n 5 -r 10 Rcf_1(f,0,1,10**5)

57.4 ms ± 7.09 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [24]:
%timeit -n 5 -r 10 Rcf_2(f,0,1,10**5)

58.2 ms ± 5.13 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [25]:
%timeit -n 5 -r 10 Rcf_3(f,0,1,10**5)

59.5 ms ± 3.08 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [26]:
%timeit -n 5 -r 10 Rcf_4(0,1,10**5)

53.1 ms ± 1.32 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


## %time

In [27]:
%time Rcf_1(f,0,1,10**5)

CPU times: user 70 ms, sys: 0 ns, total: 70 ms
Wall time: 76.1 ms


0.7468241328154887

In [28]:
%time Rcf_2(f,0,1,10**5)

CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 118 ms


0.7468241328154887

In [29]:
%time Rcf_3(f,0,1,10**5)

CPU times: user 90 ms, sys: 0 ns, total: 90 ms
Wall time: 86.9 ms


0.7468241328154887

In [30]:
%time Rcf_4(0,1,10**5)

CPU times: user 90 ms, sys: 0 ns, total: 90 ms
Wall time: 84.3 ms


0.7468241328154887

## Uso de memory_profiler: %memit y %mprun

In [31]:
%load_ext memory_profiler

In [32]:
%memit #how much RAM this process is consuming

peak memory: 97.20 MiB, increment: 0.00 MiB


In [33]:
%memit Rcf_1(f,0,1,10**5)

peak memory: 99.00 MiB, increment: 1.80 MiB


In [34]:
%memit

peak memory: 97.51 MiB, increment: 0.00 MiB


In [35]:
%memit Rcf_2(f,0,1,10**5)

peak memory: 97.51 MiB, increment: 0.00 MiB


In [36]:
%%file Rcf_1_c.py

def Rcf_1(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_1 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=[mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res 


Writing Rcf_1_c.py


In [37]:
from Rcf_1_c import Rcf_1 as Rcf_1_c

In [38]:
%mprun -f Rcf_1_c Rcf_1_c(f,0,1,10**5)

Filename: /datos/MNO_desde_2018/ramas_repo/mno-master/temas/I.computo_cientifico/Rcf_1_c.py

Line #    Mem usage    Increment   Line Contents
     2     97.5 MiB     97.5 MiB   def Rcf_1(f,a,b,n):
     3                                 """
     4                                 Compute numerical approximation using rectangle or mid-point method in 
     5                                 an interval.
     6                                 Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     7                                 Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     8                                 Args:
     9                                     f (lambda expression): lambda expression of integrand
    10                                     a (int): left point of interval
    11                                     b (int): right point of interval
    12                                     n (int): number 

## Uso de cProfile

In [39]:
%%file Rcf_1_a.py
import math
def Rcf_1(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_1 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=[mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res 
if __name__=="__main__":
        f=lambda x: math.exp(-x**2)
        print(Rcf_1(f,0,1,10**5))

Writing Rcf_1_a.py


In [40]:
%%bash
python3 -m cProfile -s cumulative Rcf_1_a.py

0.7468241328154887
         300067 function calls in 0.091 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.091    0.091 {built-in method builtins.exec}
        1    0.003    0.003    0.091    0.091 Rcf_1_a.py:1(<module>)
        1    0.014    0.014    0.088    0.088 Rcf_1_a.py:2(Rcf_1)
        1    0.030    0.030    0.042    0.042 Rcf_1_a.py:18(<listcomp>)
   100000    0.023    0.000    0.032    0.000 Rcf_1_a.py:24(<lambda>)
   100000    0.011    0.000    0.011    0.000 Rcf_1_a.py:16(<lambda>)
   100000    0.008    0.000    0.008    0.000 {built-in method math.exp}
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:966(_find_and_load)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:936(_find_and_load_unlocked)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:651(_load_unlocked)
        1    0.000    0.000    0.

**Ejercicios**

1. Resuelve los ejercicios y preguntas de la nota.


**Referencias**

1.